# 데이터 로드

In [1]:
import pandas as pd

In [2]:
review_url = 'https://raw.githubusercontent.com/kimbyeolhee/Steam-review/main/prototype/prototype_review.txt'
raw_df = pd.read_csv(review_url, sep='\t')
raw_df.sample(3)

,리뷰,플레이타임,유용한 평가,재밌는 평가,구매게임 수,작성 년도,작성 월,label
15461,어떻게 8년이 지났는데 얼엑,29.2,0,0,225.0,2020,9,0
12992,주인공 태도 존내 답답하고 짜증남스토리도 이게 끝임? 소리나옴게임플레이는 괜찮음평작...,37.8,3,0,313.0,2021,6,0
16031,어떻게 정식 출시를 한 후에 얼리억세스 상태보다 더 거지 같아 질 수 있는지... ...,232.7,1,0,98.0,2019,2,0


# 데이터 전처리

In [3]:
review = raw_df['리뷰'].copy()
review = pd.DataFrame(review)
pd.set_option('display.max_rows' ,100000)

In [4]:
review.sample(5)

,리뷰
17292,정말 재밌고 알찬 게임이네요 100시간이 아깝지 않을정도로 재밌어요^^
303,와 스컬!
11486,음~쏘 굳~~~~
9913,그래픽 음악 게임성 가격 어느하나도 부족함이 없는 게임!!
11566,"It was a simple control, but it fascinated me...."


### 중복되는 리뷰 정제

In [5]:
print(review['리뷰'].nunique())
print(len(review))

16405
18232


In [6]:
# 중복인 내용이 있다면 중복 제거
review.drop_duplicates(subset=['리뷰'], inplace=True) 
print('총 리뷰 수 :',len(review))

총 리뷰 수 : 16406


### NaN값 정제

In [7]:
print(review.isnull().values.any())

True


In [8]:
review = review.dropna(axis=0)

In [9]:
print('총 리뷰 수 :',len(review))

총 리뷰 수 : 16405


In [10]:
review = review.reset_index(drop=True)

In [11]:
review.sample()

,리뷰
3554,Good!


### 반복되는 문자 정제

In [12]:
!pip install soynlp

In [13]:
from soynlp.normalizer import *

for i in range(len(review)):
  review['리뷰'][i] = repeat_normalize(review['리뷰'][i], num_repeats=3)

In [14]:
pd.set_option('max_colwidth',1000)
review.sample(5)

,리뷰
10272,점점 빠져들어서 계속 하게됨 ㅎ.ㅎ
7670,시간 가는줄 모를 수 있음주의
1743,아 내손은 생각보다 많이 구리구나
5772,점프킹인지 추락킹인지 모르겠네 **겜
14479,갑자기 멀티로 플레이 하려면 돈을 내랍니다 미쳤나요?


### 토크나이징

In [15]:
pip install konlpy

In [16]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.


In [17]:
#import konlpy
#konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=1024)

In [18]:
from konlpy.tag import Mecab
tagger = Mecab()

In [19]:
review['토큰'] = 0

for i in range(len(review)):
  review['토큰'][i] = tagger.morphs(review['리뷰'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [20]:
review.sample(5)

,리뷰,토큰
13900,정식 나오면 쳐사세여... 제가 이겜을 3년 전에 샀음 발전은 거의 없는거 같음,"[정식, 나오, 면, 쳐, 사, 세여, ., .., 제, 가, 이, 겜, 을, 3, 년, 전, 에, 샀, 음, 발전, 은, 거의, 없, 는, 거, 같, 음]"
584,나는 무적이고 연금술사는 신이다.아ㅋㅋ 정령 45퍼들고 정령맨 하라고 ㅋㅋ,"[나, 는, 무적, 이, 고, 연금술사, 는, 신, 이, 다, ., 아, ㅋㅋ, 정령, 45, 퍼, 들, 고, 정령, 맨, 하, 라고, ㅋㅋ]"
3411,"3시간도 안되서 마지막 스테이지를 처음으로 깻을때는 뭐이리 싱겁다고 생각했었는데이후에 계~~속 이어지는 끝없는 스토리진행에 관심이 생겨 여러번 하게되고 그로인해 생기는 변화와 해금들을 보며왜 압도적으로 긍정적인지를 알게됬습니다.개성넘치는 등장인물들과의 상호작용과 호감도를 쌓으며 얻어가는 콜렉션등을 보면서아직도 해야할게 산더미라는것을 다시 한번 깨닫게 될정도니 가성비로 따지면 엄청나네요두사가 정말 귀엽습니다도전과제 전부 달성하고 느낀점은 정말 디테일 한게 장점이라고 생각합니다. 종류별로 한번씩 죽을때마다 새로 나오는 대사, 어느 무기,장신구,보조아이템 등등 죄다 다양한 것들을 들고가서 상호작용하는 캐릭터에 따라 나오는 대사, 도감을 보고 반응하는 자그레우스+이에 대한 상호작용 대사에 심지어 돌맹이까지 상호작용이 있는 줄몰랐는데 덕분에 좀더 게임을 즐기게 됬습니다. 스토리 완결 이후에도 이렇게나 많이 했는데 새로운 대사가 계속 나오는 걸보면 진짜 이만큼 공들인 작품이 있을까 싶네요.","[3, 시간, 도, 안, 되, 서, 마지막, 스테이지, 를, 처음, 으로, 깻을때는, 뭐, 이리, 싱겁, 다고, 생각, 했었, 는데, 이후, 에, 계, ~~, 속, 이어지, 는, 끝, 없, 는, 스토리, 진행, 에, 관심, 이, 생겨, 여러, 번, 하, 게, 되, 고, 그, 로, 인해, 생기, 는, 변화, 와, 해금, 들, 을, 보, 며, 왜, 압도, 적, 으로, 긍정, 적, 인지, 를, 알, 게, 됬, 습니다, ., 개성, 넘치, 는, 등장인물, 들, 과, 의, 상호, 작용, 과, 호, 감도, 를, 쌓, 으며, 얻, 어, 가, 는, 콜렉션, 등, 을, 보, 면서, 아직, 도, 해야, 할, 게, 산더미, 라는, 것, 을, 다시, ...]"
8427,"아직 별로 안한것같지만 상당히 재밌는 게임입니다장점부터 말하면 맨 처음에는 뭐 이런 불친절한 게임이 다있어 라고 할수있는데 사실 전부 다 시각(표지판 등) 청각(아이템이 근처에있을때 사운드, 애벌레 사운드, 흔들리는땅 등)에 대한 힌트가 주어지게 되어있으며 그사실을 알면 되면 맵을 이동하는 동안 탐험하는 느낌을 더욱 생동감 있게 느끼게 됩니다그리고 레벨 디자인이 탁월합니다.아직 못가는 지역은 특정 스킬을 얻어야 갈수있다거나 와 이걸 어떻게 깨지 라고 생각하는 보스들이 몇번 하다보면 어? 깨겟는데 라는 생각으로 몇시간씩 하고있는 자신의 모습을 발견하게됩니다. 불합리한 보스들이 별로 없습니다.맵 디자인 매우 좋습니다 맵고유의 색이 있어서 맵의 구분이 쉽고 맵이 유기적으로 이어져있습니다. 한 지역을 가는 길이 초반에는 한군데지만 점차 탐험할수록 2~3개 많게는 4개정도 까지되어 있습니다. 플레이어의 선택에 따라 탐험을 하는데 그것이 게임진행에 있어서 전혀 방해가 안됩니다. 플레이어마다 다른길로 가도 엔딩을 볼수있다는 맵디자인은 항상 소름 돋는것 같습니다. 맵과 보스의 브금도 좋습니다. 이건 따로 설명은 안드리겠습니다.마지막으로 가성비에 있습니다.매트로베니아 게임 특성상 플레이 타임이 많이 나오는데 여타 다른 매트로베니아 게임에 비교하여도 이가격에 초회차 시간이 말도 안되게 차이가 납니다. 얼마전에 다른 매트로베니아 게임을 하였는데 이게임의 2배정도의 가격인데 플탐이 10시간정도 나와서 자연스럽게 할나랑 비교하게되었습니다.단점으로는 아무래도 맵 시스템이 실시간 반영이 아니라 다시 의자에 가야 한다는 점인데 저도 초회차때는 불편하였으나 지금은 맵을 다 외워버려서... 단점이 아니게 되어 반론?을 하자면 코니퍼를 발견하기위해 미지의 맵을 탐험하기위해 플레이어가 긴장감을 느끼게 하여 탐험이 좀더 긴장감있게 해줄려고 하였다고 개발자의 의도를 파악해보지만...초회차때는 정말 보스보다 길이 더 어려운것은 사실입니다. 그래도 게임내 아이템들로 어느정도는 커버 가능합니다.몇...","[아직, 별로, 안, 한, 것, 같, 지만, 상당히, 재밌, 는, 게임, 입니다, 장점, 부터, 말, 하, 면, 맨, 처음, 에, 는, 뭐, 이런, 불, 친절, 한, 게임, 이, 다, 있, 어, 라고, 할, 수, 있, 는데, 사실, 전부, 다, 시각, (, 표지판, 등, ), 청각, (, 아이템, 이, 근처, 에, 있, 을, 때, 사운드, ,, 애벌레, 사운드, ,, 흔들리, 는, 땅, 등, ), 에, 대한, 힌트, 가, 주어지, 게, 되, 어, 있, 으며, 그, 사실, 을, 알, 면, 되, 면, 맵, 을, 이동, 하, 는, 동안, 탐험, 하, 는, 느낌, 을, 더욱, 생동감, 있, 게, 느끼, 게, 됩니다, 그리고, 레벨, ...]"
13797,나락,[나락]


### 불용어 제거

불용어 리스트 생성

In [21]:
token_list = []
for i in range(len(review)):
  for j in range(len(review['토큰'][i])):
    token_list.append(review['토큰'][i][j])

In [22]:
from collections import Counter
c = Counter(token_list)
print(c.most_common(500)) 

[('.', 28249), ('이', 22179), ('는', 22071), ('하', 18607), ('고', 14010), ('을', 12793), ('게임', 12256), ('가', 10841), ('은', 9898), ('에', 9682), ('다', 9659), ('도', 8989), ('있', 8951), (',', 7236), ('게', 7086), ('지', 6790), ('한', 6321), ('를', 6295), ('의', 6140), ('들', 5983), ('면', 5396), ('어', 4321), ('같', 4244), ('?', 4101), ('없', 3943), ('습니다', 3941), ('음', 3871), ('실크', 3864), ('수', 3847), ('!', 3761), ('송', 3760), ('으로', 3758), ('되', 3730), ('할', 3711), ('기', 3665), ('만', 3664), ('-', 3648), ('좋', 3628), ('거', 3624), ('것', 3607), ('로', 3569), ('안', 3544), ('는데', 3511), ('멀티', 3412), ('보', 3412), ('해', 3313), ('적', 3074), ('재밌', 2988), ('지만', 2898), ('나', 2858), ('겜', 2839), ('않', 2746), ('좀', 2707), ('아', 2604), ('에서', 2516), ('사', 2447), ('시간', 2404), ('플레이', 2328), ('핵', 2272), ('너무', 2267), ('했', 2227), ('었', 2214), ('죽', 2194), ('/', 2075), ('주', 2001), ('과', 1952), ('..', 1931), ('인', 1919), ('때', 1849), ('마', 1819), ('(', 1810), ('겠', 1796), (')', 1776), ('함', 1720), ('와', 1688), (

In [23]:
stop_words = ['이','을','에','가','를','도','의','들','은','라며','는','하고','로', '수', '것', '!', '...', '..','하는', '!!', '할', '으로', '다', '에서', '☐', '만', '때', '입니다','하면' ]

In [24]:
review['불용어 처리'] = 0

for i in range(len(review)):
  clean_words = []
  for w in review['토큰'][i]:
    if w not in stop_words:
      clean_words.append(w)
  review['불용어 처리'][i] = clean_words

review


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# 속성 개념 정의

그래픽
플레이
사운드
난이도
스토리
버그

# 속성 개념 유사어 추출

In [29]:
tokenized_data = []

for sentence in review['리뷰']:
    temp_X = tagger.nouns(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    tokenized_data.append(temp_X)

In [31]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [49]:
print(model.wv.most_similar('그래픽', topn=150))

[('도트', 0.9795791506767273), ('음악', 0.9670389294624329), ('장점', 0.9576719999313354), ('액션', 0.9575470685958862), ('분위기', 0.9530929327011108), ('스토리', 0.9491596817970276), ('매력', 0.948861837387085), ('배경', 0.9462785720825195), ('사운드', 0.9460253119468689), ('연출', 0.9429514408111572), ('라이트', 0.9425595998764038), ('로그', 0.9395804405212402), ('몰입', 0.9390907287597656), ('장르', 0.937973141670227), ('로그라이크', 0.935158371925354), ('타격', 0.932538628578186), ('느낌', 0.9245145916938782), ('편', 0.9213865995407104), ('포머', 0.9188563227653503), ('조작', 0.9148486256599426), ('메트로', 0.9146245718002319), ('브금', 0.9091275334358215), ('흥미', 0.904267430305481), ('스타일', 0.8940088748931885), ('특유', 0.893161416053772), ('세계관', 0.8928964138031006), ('단점', 0.8887576460838318), ('스크롤', 0.8886010050773621), ('개인', 0.8881800770759583), ('자체', 0.8866254091262817), ('맛', 0.8861854076385498), ('무엇', 0.8830462694168091), ('플랫', 0.8802817463874817), ('단순', 0.8801718950271606), ('독특', 0.8800051212310791), ('감성', 0.8773889

In [50]:
print(model.wv.most_similar('플레이', topn=150))

[('타임', 0.9625091552734375), ('정도', 0.9441969394683838), ('엔딩', 0.9140734672546387), ('킬링', 0.8828392624855042), ('개인', 0.8770424723625183), ('회차', 0.8704498410224915), ('볼륨', 0.8614848852157593), ('마음', 0.8606571555137634), ('확보', 0.859955906867981), ('만큼', 0.8596171140670776), ('생각', 0.8586905002593994), ('편', 0.8555934429168701), ('만족', 0.852044939994812), ('재미', 0.8512117862701416), ('노가다', 0.8504338264465332), ('흥미', 0.8482134342193604), ('클리어', 0.8465644121170044), ('완성도', 0.8462809920310974), ('제게는', 0.8436263799667358), ('이유', 0.8430149555206299), ('최근', 0.8424620032310486), ('판당', 0.8412962555885315), ('분량', 0.8412640690803528), ('완성', 0.8373146057128906), ('합의', 0.8369961977005005), ('반복', 0.8342660665512085), ('순정', 0.8332498073577881), ('입문', 0.8332258462905884), ('라이트', 0.833071231842041), ('끝', 0.8320559859275818), ('몰입', 0.8308812379837036), ('이후', 0.8293660283088684), ('리어', 0.8287689685821533), ('과제', 0.8280215263366699), ('내외', 0.826643168926239), ('계속', 0.82638508081

In [51]:
print(model.wv.most_similar('사운드', topn=150))

[('특유', 0.9861754179000854), ('세계관', 0.9855292439460754), ('브금', 0.9837518334388733), ('몰입', 0.9816357493400574), ('독특', 0.9812004566192627), ('흥미', 0.9780889749526978), ('무엇', 0.9774871468544006), ('스타일', 0.976001501083374), ('단순', 0.9754166007041931), ('분위기', 0.9752418994903564), ('감성', 0.9741643667221069), ('배경', 0.9738260507583618), ('스크롤', 0.9735677242279053), ('편안', 0.9732807278633118), ('나름', 0.9728256464004517), ('베니', 0.9719828963279724), ('로그', 0.9718652367591858), ('어드벤처', 0.9710903763771057), ('맛', 0.9710575342178345), ('손맛', 0.9709692001342773), ('매력', 0.970893144607544), ('리쉬', 0.9705855846405029), ('루팅', 0.9700219631195068), ('몽환', 0.9699156284332275), ('텔링', 0.9697151780128479), ('표방', 0.9695153832435608), ('비교', 0.9694294333457947), ('연출', 0.968543529510498), ('작화', 0.9680986404418945), ('맘', 0.9680324792861938), ('구성', 0.9672009348869324), ('뭔가', 0.9664987325668335), ('경음', 0.9662251472473145), ('감탄', 0.9652249813079834), ('횡', 0.9647631645202637), ('조작', 0.9647271633

In [42]:
print(model.wv.most_similar('난이도', topn=150))

[('소요', 0.9549081325531006), ('느낌', 0.954246997833252), ('디자인', 0.9488992691040039), ('매력', 0.9477158784866333), ('분위기', 0.9474362134933472), ('맛', 0.9436006546020508), ('회차', 0.9414464831352234), ('방식', 0.9384608268737793), ('브금', 0.9372531175613403), ('타격', 0.9366956949234009), ('탐험', 0.9356356859207153), ('클리어', 0.9353835582733154), ('배경', 0.9332325458526611), ('구성', 0.9320483803749084), ('리어', 0.9296584129333496), ('초반', 0.92836594581604), ('개성', 0.9282225966453552), ('요소', 0.9257155060768127), ('전체', 0.9257001876831055), ('전투', 0.9255492687225342), ('목적지', 0.9254082441329956), ('편', 0.9240280389785767), ('흥미', 0.9236564636230469), ('특성', 0.9226653575897217), ('단순', 0.9225465059280396), ('음악', 0.9217633008956909), ('스타일', 0.9214688539505005), ('데드', 0.9213854670524597), ('레벨', 0.9196058511734009), ('반복', 0.9195955991744995), ('조작', 0.9193858504295349), ('단점', 0.9169657230377197), ('셀', 0.9163287281990051), ('연출', 0.9157592058181763), ('중간', 0.91098952293396), ('수집', 0.9104483127593

In [43]:
print(model.wv.most_similar('스토리', topn=150))

[('느낌', 0.9743914604187012), ('연출', 0.9722648859024048), ('음악', 0.9716437458992004), ('장점', 0.964500904083252), ('편', 0.960515558719635), ('매력', 0.9584064483642578), ('몰입', 0.9528172612190247), ('배경', 0.9528138041496277), ('그래픽', 0.9491596817970276), ('분위기', 0.948351263999939), ('단점', 0.9402166604995728), ('흥미', 0.934740424156189), ('사운드', 0.9337632060050964), ('라이트', 0.933449923992157), ('로그', 0.9294800758361816), ('자체', 0.9252108335494995), ('부분', 0.9243652820587158), ('도트', 0.9180371761322021), ('방식', 0.9171670079231262), ('브금', 0.9164894819259644), ('요소', 0.912967324256897), ('세계관', 0.9106883406639099), ('장르', 0.9081563949584961), ('진행', 0.9075584411621094), ('무엇', 0.9073613882064819), ('액션', 0.9053348302841187), ('구색', 0.9046597480773926), ('개인', 0.902890682220459), ('전체', 0.900144100189209), ('타격', 0.8998036980628967), ('난이도', 0.896473228931427), ('맛', 0.8961275815963745), ('단순', 0.8960371017456055), ('루팅', 0.8939991593360901), ('독특', 0.8930695056915283), ('나름', 0.892928242683410

In [45]:
print(model.wv.most_similar('버그', topn=150))

[('발매', 0.915583610534668), ('해결', 0.8990024328231812), ('정시', 0.898905336856842), ('개발', 0.8889597058296204), ('수정', 0.8733413219451904), ('팀', 0.8629262447357178), ('수년', 0.8620221018791199), ('정식', 0.854534387588501), ('월', 0.8502141833305359), ('후속작', 0.8499106168746948), ('버전', 0.8491921424865723), ('서버', 0.8485347032546997), ('베타', 0.8484337329864502), ('공지', 0.8484045267105103), ('최신', 0.8480952382087708), ('출시', 0.8471933007240295), ('안티', 0.8455398082733154), ('인터뷰', 0.8427509069442749), ('오후', 0.8419438600540161), ('오류', 0.8417012691497803), ('초창기', 0.8414919376373291), ('진', 0.8408326506614685), ('감사', 0.8384226560592651), ('상태', 0.8366515636444092), ('공식', 0.8363609313964844), ('표', 0.8356033563613892), ('픽스', 0.8346772193908691), ('한국어', 0.8332419395446777), ('자막', 0.8324817419052124), ('벤트', 0.8322518467903137), ('프레임', 0.8317948579788208), ('예정', 0.8289110660552979), ('개선', 0.8279021978378296), ('후자', 0.8249021768569946), ('현기증', 0.8247727751731873), ('먹통', 0.82427394390

In [56]:
# 속성 개념 유사어 리스트 생성

###그래픽
graphic = []
for i in range(150):
  graphic.append(model.wv.most_similar('그래픽', topn=150)[i][0])

###플레이
play = []
for i in range(150):
  play.append(model.wv.most_similar('플레이', topn=150)[i][0])

###사운드
sound = []
for i in range(150):
  sound.append(model.wv.most_similar('사운드', topn=150)[i][0])

###난이도
level = []
for i in range(150):
  level.append(model.wv.most_similar('난이도', topn=150)[i][0])

###스토리
story = []
for i in range(150):
  story.append(model.wv.most_similar('스토리', topn=150)[i][0])

###버그
bug = []
for i in range(150):
  bug.append(model.wv.most_similar('버그', topn=150)[i][0])

개념 속성 유사어 리스트 텍스트파일로 저장

In [58]:
with open('그래픽_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in graphic:
    f.write(w+'\n')

In [59]:
with open('플레이_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in play:
    f.write(w+'\n')

In [60]:
with open('사운드_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in sound:
    f.write(w+'\n')

In [61]:
with open('난이도_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in level:
    f.write(w+'\n')

In [62]:
with open('스토리_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in story:
    f.write(w+'\n')

In [63]:
with open('버그_유사속성어.txt', 'w', encoding='UTF-8') as f:
  for w in bug:
    f.write(w+'\n')